# Estandarizacion de datos del Inventario Nacional de Energias Renovables
## 1. Introduccion
Parámetros que salen de esta fuente

ID |Descripción
---|:----------
P0009|Potencial de aprovechamiento energía solar
P0010|Potencial de aprovechamiento energía eólica
P0011|Potencial de aprovechamiento energía geotérmica
P0012|Potencial de aprovechamiento energía de biomasa
P0606|Generación mediante fuentes renovables de energía
P0607|Potencial de fuentes renovables de energía
P0608|Capacidad instalada para aprovechar fuentes renovables de energía

In [21]:
descripciones = {
'P0009' : 'Potencial de aprovechamiento energía solar',
'P0010' : 'Potencial de aprovechamiento energía eólica',
'P0011' : 'Potencial de aprovechamiento energía geotérmica',
'P0012' : 'Potencial de aprovechamiento energía de biomasa',
'P0606' : 'Generación mediante fuentes renovables de energía',
'P0607' : 'Potencial de fuentes renovables de energía',
'P0608' : 'Capacidad instalada para aprovechar fuentes renovables de energía'
}

In [22]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib
import os
import csv

In [23]:
# Configuracion del sistema
print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


## Descarga de datos

Los datos se encuentran en la plataforma del Inventario Nacional de Energias Renovables (INERE) ubicada en https://dgel.energia.gob.mx/inere/, y tienen que descargarse manualmente porque su página está elaborada en Flash y no permite la descarga sistematizada de datos. A veces ni funciona por sí misma, manda errores al azar.

Se descargaron dos datasets, uno que contiene el **Inventario Actual** y otro con el **Inventario Potencial** de energías renovables a nivel nacional.

Como la base de datos no incluye claves geoestadísticas, estas tienen que asignarse manualmente. A continuacion se muestra el encabezado del archivo que se procesó a mano.

In [4]:
# Lectura del dataset de energia renovable actual como descargado
directorio = r'D:\PCCS\00_RawData\01_CSV\INERE\\'
archivo = directorio+'Actual Energia Renovable.xls'
raw_actual = pd.read_excel(archivo).dropna()
raw_actual.head()
# El dataset envía error cuando se intenta leer directamente

XLRDError: Unsupported format, or corrupt file: BOF not workbook/worksheet: op=0x0009 vers=0x0000 strm=0x1000 build=0 year=0 -> BIFF21

Ninguno de los dos datasets puede ser leido por python tal como fue descargado, por lo que tienen que abrirse en excel y guardarse nuevamente en formato xlsx.

### Dataset energia renovable actual

In [24]:
# Lectura del dataset de energia renovable actual después de ser re-guardado en excel
directorio = r'D:\PCCS\00_RawData\01_CSV\INERE\\'
archivo = directorio+'Actual Energia Renovable.xlsx'
raw_actual = pd.read_excel(archivo).dropna()
raw_actual.head()

,ESTADO,MUNICIPIO,NOMBRE,PRODUCTOR,TIPO,UNIDADES,CAPACIDAD INSTALADA (MW),GENERACIÓN (GWh/a)
1,Querétaro,Santiago de Querétaro,Atlatec,Privado,Biogás,1.0,1.049,3.824943
2,Jalisco,Tlajomulco De Zuñiga,"Atlatec, Planta El Ahogado",Privado,Biogás,2.0,2.830,6.918000
3,Tabasco,Tenosique,Azsuremex,Privado,Combustión directa,2.0,2.500,2.923360
4,Jalisco,Tuxpan,"Bio Pappel, Atenquique",Privado,Combustión directa,4.0,15.500,32.240000
5,Nuevo León,Salinas Victoria,Bioenergía de Nuevo León,Privado,Biogás,7.0,16.960,87.759000


Se asignó CVE_MUN manualmente a la mayoría de los registros. No fue posible encontrar una clave geoestadística para las siguientes combinaciones de estado/municipio 

ESTADO |MUNICIPIO
-------|:----------
Veracruz|Jiotepec
Chiapas|Atizapan
Oaxaca|Motzorongo
Guerrero|La Venta
Jalisco|Santa Rosa

Para los siguientes registros, la CVE_MUN fue intuida desde el nombre de la población o el nombre del proyecto:

ESTADO |MUNICIPIO|CVE_MUN|PROYECTO
-------|:----------|-------|------
Puebla|Atencingo|21051|Ingenio de Atencingo
Puebla|Tatlahuquitepec|21186|Mazatepec

A continuación se presenta el encabezado del dataset procesado manualmente, incluyendo columnas que se utilizaron como auxiliares para la identificación de municipios


In [25]:
# Lectura del dataset de energia renovable actual procesado manualmente
directorio = r'D:\PCCS\00_RawData\01_CSV\INERE\\'
archivo = directorio+'Actual CVE_GEO.xlsx'
actual_proc = pd.read_excel(archivo, dtype={'CVE_MUN': 'str'}).dropna()
actual_proc.head()

,ESTADO,MUNICIPIO,3EDO3,3MUN3,GEO_EDO,GEOEDO_3MUN,CVE_MUN,GEO_MUN_Nom,NOMBRE,PRODUCTOR,TIPO,UNIDADES,CAPACIDAD INSTALADA (MW),GENERACIÓN (GWh/a)
0,Querétaro,Santiago de Querétaro,QUEARO,SANARO,22,22SANARO,22014,QUERÉTARO_Querétaro,Atlatec,Privado,Biogás,1,1.049,3.824943
1,Jalisco,Tlajomulco De Zuñiga,JALSCO,TLAIGA,14,14TLAIGA,14097,JALISCO_Tlajomulco de Zúñiga,"Atlatec, Planta El Ahogado",Privado,Biogás,2,2.830,6.918000
2,Tabasco,Tenosique,TABSCO,TENQUE,27,27TENQUE,27017,TABASCO_Tenosique,Azsuremex,Privado,Combustión directa,2,2.500,2.923360
3,Jalisco,Tuxpan,JALSCO,TUXPAN,14,14TUXPAN,14108,JALISCO_Tuxpan,"Bio Pappel, Atenquique",Privado,Combustión directa,4,15.500,32.240000
4,Nuevo León,Salinas Victoria,NUEEÓN,SALRIA,19,19SALRIA,19045,NUEVO LEÓN_Salinas Victoria,Bioenergía de Nuevo León,Privado,Biogás,7,16.960,87.759000


Para guardar el dataset y utilizarlo en la construcción del parámetro, se eliminarán algunas columnas.

In [26]:
list(actual_proc)

['ESTADO',
 'MUNICIPIO',
 '3EDO3',
 '3MUN3',
 'GEO_EDO',
 'GEOEDO_3MUN',
 'CVE_MUN',
 'GEO_MUN_Nom',
 'NOMBRE',
 'PRODUCTOR',
 'TIPO',
 'UNIDADES',
 'CAPACIDAD INSTALADA (MW)',
 'GENERACIÓN (GWh/a) ']

In [27]:
# Eliminacion de columnas redundantes y temporales
del(actual_proc['ESTADO'])
del(actual_proc['MUNICIPIO'])
del(actual_proc['3EDO3'])
del(actual_proc['3MUN3'])
del(actual_proc['GEO_EDO'])
del(actual_proc['GEOEDO_3MUN'])
del(actual_proc['GEO_MUN_Nom'])

# Nombre Unico de Coloumnas
actual_proc = actual_proc.rename(columns = {
    'NOMBRE' : 'NOMBRE PROYECTO', 
    'PRODUCTOR': 'SECTOR PRODUCCION', 
    'TIPO': 'TIPO FUENTE ENER', 
    'UNIDADES': 'UNIDADES GEN'})

# Asignacion de CVE_MUN como indice 
actual_proc.set_index('CVE_MUN', inplace=True)
actual_proc.head()

,NOMBRE PROYECTO,SECTOR PRODUCCION,TIPO FUENTE ENER,UNIDADES GEN,CAPACIDAD INSTALADA (MW),GENERACIÓN (GWh/a)
CVE_MUN,,,,,,
22014,Atlatec,Privado,Biogás,1,1.049,3.824943
14097,"Atlatec, Planta El Ahogado",Privado,Biogás,2,2.830,6.918000
27017,Azsuremex,Privado,Combustión directa,2,2.500,2.923360
14108,"Bio Pappel, Atenquique",Privado,Combustión directa,4,15.500,32.240000
19045,Bioenergía de Nuevo León,Privado,Biogás,7,16.960,87.759000


In [28]:
# Metadatos estándar
metadatos = {
    'Nombre del Dataset': 'Inventario Actual de Energias Renovables',
    'Descripcion del dataset': 'Plantas de generación de energía a partir de fuentes renovables en la República Mexicana',
    'Disponibilidad Temporal': '2014',
    'Periodo de actualizacion': 'No Determinada',
    'Nivel de Desagregacion': 'Localidad, Municipal, Estatal, Nacional',
    'Notas': None,
    'Fuente': 'SENER',
    'URL_Fuente': 'https://dgel.energia.gob.mx/inere/',
    'Dataset base': None,
}

# Convertir metadatos a dataframe
actualmeta = pd.DataFrame.from_dict(metadatos, orient='index', dtype=None)
actualmeta.columns = ['Descripcion']
actualmeta = actualmeta.rename_axis('Metadato')
actualmeta

,Descripcion
Metadato,
Nombre del Dataset,Inventario Actual de Energias Renovables
Descripcion del dataset,Plantas de generación de energía a partir de f...
Disponibilidad Temporal,2014
Periodo de actualizacion,No Determinada
Nivel de Desagregacion,"Localidad, Municipal, Estatal, Nacional"
Notas,None
Fuente,SENER
URL_Fuente,https://dgel.energia.gob.mx/inere/
Dataset base,None


In [29]:
list(actual_proc)

['NOMBRE PROYECTO',
 'SECTOR PRODUCCION',
 'TIPO FUENTE ENER',
 'UNIDADES GEN',
 'CAPACIDAD INSTALADA (MW)',
 'GENERACIÓN (GWh/a) ']

In [30]:
# Descripciones de columnas
variables = {
    'NOMBRE PROYECTO': 'Nombre del proyecto de produccion de energia',
    'SECTOR PRODUCCION': 'Sector al que pertenece el proyecto de produccion de energia',
    'TIPO FUENTE ENER': 'Tipo de fuente de donde se obtiene la energía',
    'UNIDADES GEN': 'Numero de generadores instalados por proyecto',
    'CAPACIDAD INSTALADA (MW)': 'Capacidad Instalada en Megawatts',
    'GENERACIÓN (GWh/a) ' : 'Generación de Gigawatts/hora al año'
}

# Convertir descripciones a dataframe
actualvars = pd.DataFrame.from_dict(variables, orient='index', dtype=None)
actualvars.columns = ['Descripcion']
actualvars = actualvars.rename_axis('Mnemonico')
actualvars

,Descripcion
Mnemonico,
NOMBRE PROYECTO,Nombre del proyecto de produccion de energia
SECTOR PRODUCCION,Sector al que pertenece el proyecto de producc...
TIPO FUENTE ENER,Tipo de fuente de donde se obtiene la energía
UNIDADES GEN,Numero de generadores instalados por proyecto
CAPACIDAD INSTALADA (MW),Capacidad Instalada en Megawatts
GENERACIÓN (GWh/a),Generación de Gigawatts/hora al año


In [31]:
# Guardar dataset limpio para creacion de parametro.
file = r'D:\PCCS\01_Dmine\Datasets\INERE\ER_Actual.xlsx'
writer = pd.ExcelWriter(file)
actual_proc.to_excel(writer, sheet_name = 'DATOS')
actualmeta.to_excel(writer, sheet_name = 'METADATOS')
actualvars.to_excel(writer, sheet_name = 'VARIABLES')
writer.save()
print('---------------TERMINADO---------------')

---------------TERMINADO---------------


### Dataset Potencial de Energia Renovable

In [32]:
# Lectura del dataset de potencial de energia renovable después de ser re-guardado en excel
directorio = r'D:\PCCS\00_RawData\01_CSV\INERE\\'
archivo = directorio+'Potencial Energia Renovable.xlsx'
raw_potencial = pd.read_excel(archivo, dtype={'CVE_MUN': 'str'}).dropna()
raw_potencial.head()

,ESTADO,MUNICIPIO,3EDO3,3MUN3,GEO_EDO,GEOEDO_3MUN,CVE_MUN,GEO_MUN_Nom,PROYECTO,TIPO,CLASIFICACIÓN,SUBCLASIFICACIÓN,CAPACIDAD INSTALABLE (MW),POTENCIAL (GWh/a),FUENTE
0,Sonora,Agua Prieta,SONORA,AGUETA,26,26AGUETA,26002,SONORA_Agua Prieta,Agua Prieta II,Termosolar Parabólica,Probado,En construcción,12.000,18.86400,CFE
1,Aguascalientes,Asientos,AGUTES,ASITOS,1,01ASITOS,01002,AGUASCALIENTES_Asientos,Aguascalientes 1,Biogás,Probable,Motogenerador,1.510,8.83008,CFE
2,Aguascalientes,Asientos,AGUTES,ASITOS,1,01ASITOS,01002,AGUASCALIENTES_Asientos,Aguascalientes 2,Biogás,Probable,Motogenerador,1.512,8.83008,CFE
3,Aguascalientes,Asientos,AGUTES,ASITOS,1,01ASITOS,01002,AGUASCALIENTES_Asientos,Aguascalientes 3,Biogás,Probable,Biodigestor,3.000,17.52000,CFE
4,Aguascalientes,Jesús María,AGUTES,JESRÍA,1,01JESRÍA,01005,AGUASCALIENTES_Jesús María,Aguascalientes 4,Biogás,Probable,Biodigestor,0.093,0.54312,CFE


In [33]:
# Eliminacion de columnas redundantes y temporales
potencial_proc = raw_potencial
del(potencial_proc['ESTADO'])
del(potencial_proc['MUNICIPIO'])
del(potencial_proc['3EDO3'])
del(potencial_proc['3MUN3'])
del(potencial_proc['GEO_EDO'])
del(potencial_proc['GEOEDO_3MUN'])
del(potencial_proc['GEO_MUN_Nom'])

potencial_proc.head()

,CVE_MUN,PROYECTO,TIPO,CLASIFICACIÓN,SUBCLASIFICACIÓN,CAPACIDAD INSTALABLE (MW),POTENCIAL (GWh/a),FUENTE
0,26002,Agua Prieta II,Termosolar Parabólica,Probado,En construcción,12.000,18.86400,CFE
1,01002,Aguascalientes 1,Biogás,Probable,Motogenerador,1.510,8.83008,CFE
2,01002,Aguascalientes 2,Biogás,Probable,Motogenerador,1.512,8.83008,CFE
3,01002,Aguascalientes 3,Biogás,Probable,Biodigestor,3.000,17.52000,CFE
4,01005,Aguascalientes 4,Biogás,Probable,Biodigestor,0.093,0.54312,CFE


In [34]:
potencial_proc['SUBCLASIFICACIÓN'].unique()

array(['En construcción', 'Motogenerador', 'Biodigestor', 'Estudios',
       'Baja Entalpia', 'Mediana Entalpia', 'Alta Entalpia',
       'Por iniciar obras', 'En operación', 'Inactivo'], dtype=object)

In [35]:
# Nombre Unico de Coloumnas
potencial_proc = potencial_proc.rename(columns = {
    'PROYECTO' : 'NOMBRE PROYECTO', 
    'CLASIFICACIÓN': 'PROBABILIDAD', 
    'TIPO': 'TIPO FUENTE ENER',
    'SUBCLASIFICACIÓN': 'NOTAS'})

# Asignacion de CVE_MUN como indice 
potencial_proc.set_index('CVE_MUN', inplace=True)
potencial_proc.head()

,NOMBRE PROYECTO,TIPO FUENTE ENER,PROBABILIDAD,NOTAS,CAPACIDAD INSTALABLE (MW),POTENCIAL (GWh/a),FUENTE
CVE_MUN,,,,,,,
26002,Agua Prieta II,Termosolar Parabólica,Probado,En construcción,12.000,18.86400,CFE
01002,Aguascalientes 1,Biogás,Probable,Motogenerador,1.510,8.83008,CFE
01002,Aguascalientes 2,Biogás,Probable,Motogenerador,1.512,8.83008,CFE
01002,Aguascalientes 3,Biogás,Probable,Biodigestor,3.000,17.52000,CFE
01005,Aguascalientes 4,Biogás,Probable,Biodigestor,0.093,0.54312,CFE


In [39]:
# Metadatos estándar
metadatos = {
    'Nombre del Dataset': 'Inventario Potencial de Energias Renovables',
    'Descripcion del dataset': 'listado de Proyectos con potencial para generar energía a partir de fuentes renovables',
    'Disponibilidad Temporal': '2014',
    'Periodo de actualizacion': 'No Determinada',
    'Nivel de Desagregacion': 'Localidad, Municipal, Estatal, Nacional',
    'Notas': None,
    'Fuente': 'SENER',
    'URL_Fuente': 'https://dgel.energia.gob.mx/inere/',
    'Dataset base': None,
}

# Convertir metadatos a dataframe
potenmeta = pd.DataFrame.from_dict(metadatos, orient='index', dtype=None)
potenmeta.columns = ['Descripcion']
potenmeta = potenmeta.rename_axis('Metadato')
potenmeta

,Descripcion
Metadato,
Nombre del Dataset,Inventario Potencial de Energias Renovables
Descripcion del dataset,listado de Proyectos con potencial para genera...
Disponibilidad Temporal,2014
Periodo de actualizacion,No Determinada
Nivel de Desagregacion,"Localidad, Municipal, Estatal, Nacional"
Notas,None
Fuente,SENER
URL_Fuente,https://dgel.energia.gob.mx/inere/
Dataset base,None


In [40]:
list(potencial_proc)

['NOMBRE PROYECTO',
 'TIPO FUENTE ENER',
 'PROBABILIDAD',
 'NOTAS',
 'CAPACIDAD INSTALABLE (MW)',
 'POTENCIAL (GWh/a)',
 'FUENTE']

In [41]:
potencial_proc['FUENTE'].unique()

array(['CFE', 'UNAM', 'CMM', 'CRE', 'IMTA'], dtype=object)

In [42]:
# Descripciones de columnas
variables = {
    'NOMBRE PROYECTO': 'Nombre del proyecto de produccion de energia',
    'TIPO FUENTE ENER': 'Tipo de fuente de donde se obtiene la energía',
    'PROBABILIDAD': 'Certeza respecto al proyecto deproduccion de energía',
    'NOTAS': 'Notas',
    'CAPACIDAD INSTALABLE (MW)': 'Capacidad Instalable en Megawatts',
    'POTENCIAL (GWh/a) ' : 'Potencial de Generación de Gigawatts/hora al año',
    'FUENTE': 'Fuente de información'
}

# Convertir descripciones a dataframe
potencialvars = pd.DataFrame.from_dict(variables, orient='index', dtype=None)
potencialvars.columns = ['Descripcion']
potencialvars = potencialvars.rename_axis('Mnemonico')
potencialvars

,Descripcion
Mnemonico,
NOMBRE PROYECTO,Nombre del proyecto de produccion de energia
TIPO FUENTE ENER,Tipo de fuente de donde se obtiene la energía
PROBABILIDAD,Certeza respecto al proyecto deproduccion de e...
NOTAS,Notas
CAPACIDAD INSTALABLE (MW),Capacidad Instalable en Megawatts
POTENCIAL (GWh/a),Potencial de Generación de Gigawatts/hora al año
FUENTE,Fuente de información


In [43]:
# Guardar dataset limpio para creacion de parametro.
file = r'D:\PCCS\01_Dmine\Datasets\INERE\ER_Potencial.xlsx'
writer = pd.ExcelWriter(file)
potencial_proc.to_excel(writer, sheet_name = 'DATOS')
potenmeta.to_excel(writer, sheet_name = 'METADATOS')
potencialvars.to_excel(writer, sheet_name = 'VARIABLES')
writer.save()
print('---------------TERMINADO---------------')

---------------TERMINADO---------------
